# Evaluate Hex Chess Engines

## Setup

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
from hexchess import HexChessBoard, Player, RandomPlayer, GreedyPlayer
from engines import QNetworkPlayer

pygame 2.5.2 (SDL 2.28.2, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-03-21 15:30:12.998707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 15:30:13.156666: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-21 15:30:13.156691: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-21 15:30:13.882224: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [3]:
# Constants
results_dir = "./assets/results"
os.makedirs(results_dir, exist_ok=True)

## Game Function

In [25]:
def play_game(white_player_class, black_player_class, max_moves = 50):
    if isinstance(white_player_class, dict):
        white_player_kwargs = white_player_class['kwargs']
        white_player_class = white_player_class['class']
    else:
        white_player_kwargs = {}
    if isinstance(black_player_class, dict):
        black_player_kwargs = black_player_class['kwargs']
        black_player_class = black_player_class['class']
    else:
        black_player_kwargs = {}
    
    # Initialize board
    board = HexChessBoard()

    # Initialize players
    white_player = white_player_class(board, True, **white_player_kwargs)
    black_player = black_player_class(board, False, **black_player_kwargs)

    # Play game
    n_moves = 0
    turn_is_white = True
    finished = False
    for _ in range(max_moves * 2):
        # Select active player
        active_player = white_player if turn_is_white else black_player

        # Get move
        position_from, position_to = active_player.get_move()
        success, finished = board.move(position_from, position_to, turn_is_white)

        # Update turn
        turn_is_white = not turn_is_white
        n_moves += 1 if turn_is_white else 0
        if finished:
            break


    # Determine winner (only valid if game is finished)
    white_won = not turn_is_white
    return finished, white_won, n_moves

## Evaluate Games

In [26]:
# Configuration
n_games = 10
max_moves = 100
white_players = [
    {"class": RandomPlayer, "label": "Random (baseline)", "kwargs": {}}, 
    {"class": GreedyPlayer, "label": "Greedy (baseline)", "kwargs": {}},
    {"class": QNetworkPlayer, "label": "DQN (random)", "kwargs": {"model_name": "random_pr"}},
    {"class": QNetworkPlayer, "label": "DQN (random + greedy)", "kwargs": {"model_name": "randomgreedy_pr"}},
]
black_players = white_players

In [27]:
# Gather results
rows = [player["label"] if isinstance(player, dict) else player.name for player in white_players]
cols = [player["label"] if isinstance(player, dict) else player.name for player in black_players]
games_won = np.zeros((len(white_players), len(black_players)), dtype=int)
games_draw = np.zeros((len(white_players), len(black_players)), dtype=int)
games_lost = np.zeros((len(white_players), len(black_players)), dtype=int)
games_moves = np.zeros((len(white_players), len(black_players)), dtype=float)

# All white players
for i, white_player in enumerate(white_players):
    # Against all black players
    for j, black_player in enumerate(black_players):
        # Collect statistics
        for game_index in range(n_games):
            # Play Game
            game_finished, white_won, n_moves = play_game(white_player, black_player, max_moves=max_moves)

            # Win / Loss / Draw
            if game_finished:
                if white_won:
                    games_won[i, j] += 1
                else:
                    games_lost[i, j] += 1
            else:
                games_draw[i, j] += 1

            # Number of moves
            games_moves[i, j] += n_moves / n_games


In [29]:
# Collect results in Dataframes
games_won_df = pd.DataFrame(data = games_won, index = rows, columns = cols)
games_lost_df = pd.DataFrame(data = games_lost, index = rows, columns = cols)
games_draw_df = pd.DataFrame(data = games_draw, index = rows, columns = cols)
games_moves_df = pd.DataFrame(data = games_moves, index = rows, columns = cols)

In [30]:
# Save results
games_df_names = ["won", "lost", "draw", "moves"]
games_dfs = [games_won_df, games_lost_df, games_draw_df, games_moves_df]
for name, game_df in zip(games_df_names, games_dfs):
    game_df.to_csv(os.path.join(results_dir, f"games_{name}.csv"), index=True)

In [35]:
# Load results
games_df_names = ["won", "lost", "draw", "moves"]
games_dfs = [pd.read_csv(os.path.join(results_dir, f"games_{name}.csv"), index_col=0) for name in games_df_names]
games_won_df, games_lost_df, games_draw_df, games_moves_df = games_dfs

In [36]:
print("___GAMES WON___")
games_won_df

___GAMES WON___


,Random (baseline),Greedy (baseline),DQN (random),DQN (random + greedy)
Random (baseline),4,0,2,1
Greedy (baseline),10,9,10,9
DQN (random),4,2,0,0
DQN (random + greedy),3,2,0,0


In [37]:
print("___GAMES LOST___")
games_lost_df

___GAMES LOST___


,Random (baseline),Greedy (baseline),DQN (random),DQN (random + greedy)
Random (baseline),4,10,8,4
Greedy (baseline),0,1,0,1
DQN (random),6,8,0,0
DQN (random + greedy),5,8,10,0


In [38]:
print("___GAMES DRAW___")
games_draw_df

___GAMES DRAW___


,Random (baseline),Greedy (baseline),DQN (random),DQN (random + greedy)
Random (baseline),2,0,0,5
Greedy (baseline),0,0,0,0
DQN (random),0,0,10,10
DQN (random + greedy),2,0,0,10


In [39]:
print("__NR OF MOVES__")
games_moves_df

__NR OF MOVES__


,Random (baseline),Greedy (baseline),DQN (random),DQN (random + greedy)
Random (baseline),46.5,10.9,38.0,70.2
Greedy (baseline),9.5,19.0,10.5,19.3
DQN (random),53.4,10.0,100.0,100.0
DQN (random + greedy),51.6,15.6,5.0,100.0
